## Strategy debugging example

Debugging a strategy can be time-consuming. FreqTrade offers helper functions to visualize raw data.

## Setup

### Change directory
Jupyter notebooks execute from the current directory. Change working directory to the project root so that relative paths work.

In [ ]:
import os
from pathlib import Path

# Modify this cell to insure that the output shows the correct path.
project_root = "somedir/freqtrade"
i = 0
try:
    os.chdirdir(project_root)
    assert Path('LICENSE').is_file()
except:
    while i < 4 and (not Path('LICENSE').is_file()):
        os.chdir(Path(Path.cwd(), '../'))
        i += 1
    project_root = Path.cwd()
print(Path.cwd())

### Fix asycronous execution
Jupyter notebooks get confused with async operations. This fixes the problem.

In [ ]:
import nest_asyncio

# Fix asyncio for Jupyter
nest_asyncio.apply()

### Generate configuration
This combines all config files into a single dictionary. Individual options can be manipulated with dict indexing.
Watch out for type conversion pitfalls. The config dict created here has string values. More work needs to be done to automatically coerce strings to the expected object types. You will need to perform type conversion when passing function arguments as demonstrated in the following sections.

In [ ]:
from freqtrade.configuration import Configuration
from freqtrade.configuration.timerange import TimeRange

# Load configuration
# Specify values for use in this script
############### Customize to match your needs. ##################
config_files = [
    Path('user_data', 'user_repo', 'config.json'),
    Path(Path.home(), '.freqtrade', 'exchange-config.json')
]
# Create config object
config = Configuration.from_files(config_files)

############### Customize to match your needs. ##################
# Define some constants

# These values must be included in config.json or set here
# Name of the strategy class
config['strategy'] = 'DefaultStrategy'
# Path to user data
config['user_data_dir'] = Path('user_data')
# Location of the ticker data
config['datadir'] = Path('user_data', 'data/binance')
# Location of the strategy
config['strategy_path'] = Path('user_data', 'strategies')
# Specify backtest results to load
config['trade_source'] = 'file'
config['exportfilename'] = Path('user_data', 'backtest_results/backtest-result.json')
config['db_url'] = 'sqlite://'

# Specify interval to analyze
ticker_interval = "5m"
# Specify timerange to test
timerange = '-100'
# Pair to analyze - Only use one pair here
pair = "ETH/BTC"
# Indicators from strategy to plot
overlay_indicators = ['ema50', 'ema100']
bottom_indicators = ['macd']

### Generate config argument for CLI
CLI commands can be executed from a notebook cell by including `!` before the command. This combines all config files into a variable that can be called in a cli command as `{conf}`

In [ ]:
from itertools import chain

# Create config string for use in cli commands
conf = " ".join(
    list(chain.from_iterable([['-c', str(file)] for file in config_files])))

# Example cli command
!freqtrade {conf} backtesting --help

### Configure logging
This imports the freqtrade logger format and displays logging messages from the internal functions.

In [ ]:
import logging
import json
from freqtrade.loggers import setup_logging

# Configure logging
logger = logging.getLogger()
setup_logging(config)
logger.setLevel(logging.INFO)
logger.info(f'conf: {conf}')

## Visualize buy/sell signals
This uses the values set in the configuration section to:
* Load the history of the specified pair
* Load the specified strategy
* Generate buy and sell signals produced by the specified strategy
* Return a dictionary with the figure, figure dataframe and trades dataframe
    * Key is the same as the figure filename. EG: 'freqtrade-plot-ETH_BTC-15m.html'
    * Display elements with dict indexing as demonstrated below
    * `fig.show()` displays the chart inline `fig.show(renderer="browser")` displays the chart in a tab.

*Note: `data.head()` may include empty values for indicators that require a startup period. This is expected behavior. For example `ma5` requires 5 candles before computing the first average*

### Possible problems

* Columns with NaN values at the end of the dataframe
* Columns used in `crossed*()` functions with completely different units

### Comparison with full backtest

* having 200 buy signals as output for one pair from `analyze_ticker()` does not necessarily mean that 200 trades will be made during backtesting.
* Assuming you use only one condition such as, `df['rsi'] < 30` as buy condition, this will generate multiple "buy" signals for each pair in sequence (until rsi returns > 29). The bot will only buy on the first of these signals (and also only if a trade-slot ("max_open_trades") is still available), or on one of the middle signals, as soon as a "slot" becomes available.  

In [ ]:
from freqtrade.plot.plotting import load_and_plot_trades

plot_data = load_and_plot_trades({'strategy': config['strategy'],
                                    'strategy_path': Path(config['strategy_path']),
                                    'timerange': timerange,
                                    'ticker_interval': ticker_interval,
                                    'strategy_path': Path(config['strategy_path']),
                                    'datadir': Path(config['datadir']),
                                    'user_data_dir': Path(config['user_data_dir']),
                                    'exchange': config['exchange'],
                                    'trade_source': config['trade_source'],
                                    'exportfilename': config['exportfilename'],
                                    'indicators1': overlay_indicators,
                                    'indicators2': bottom_indicators
                           })

In [ ]:
# Display charts and data inline
for idx in list(plot_data.keys()):
    print(idx)
    plot_data[idx]['fig'].show()
    display(plot_data[idx]['data'].tail())
    display(plot_data[idx]['trades'].head())

### Detailed analysis
Slice and dice your data to generate insights

In [ ]:
def show_indicator_extremes(df, indicator):
    '''
    Sorts simulation dataframe by specified indicator
    '''
    df = df[df[indicator].notna()].sort_values(by=indicator, ascending=False)
    return df

# Demonstrate with first item
idx = list(plot_data.keys())[0]
indicator = overlay_indicators[0]
print(idx)
data = plot_data[idx]['data']
extremes = show_indicator_extremes(data, indicator)
display(extremes.head(10))
display(extremes.tail(10))

### Run Backtest
Once you are happy with your strategy signals, run a backtest then plot again.

Feel free to submit an issue or Pull Request enhancing this document if you would like to share ideas on how to best analyze the data.